In [8]:
from fpgrowth_py import fpgrowth
import pandas as pd
import re
import json
import spacy
import seaborn as sns
from spacy import displacy
from flashtext import KeywordProcessor
# import pyplot-express as px

pathprefix = r"D:\Code Working Area\Jupyter\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData"
jsfile = pathprefix+"\\Transactions.json"

project = pathprefix+"\\Gg.csv"
risk = pathprefix+"\\Risk_Simplified.xlsx"
stake = pathprefix+"\\expansive.csv"

project1 = pathprefix+"\\newTitle_Project.xlsx"
risk0 = pathprefix+"\\RiskFinal.xlsx"
stake1 = pathprefix+"\\New_StakeHolder_Abstract.xlsx"

stake2 = pathprefix+"\\second_layer_stakeholder.xlsx"

In [6]:
# !pip install regex
# !pip install sapcy
# !pip install fpgrowth_py
# !pip install pandas
# !pip install seaborn
# !pip install openpyxl
# !python -m spacy install en_core_web_sm
stake2

'D:\\Code Working Area\\Jupyter\\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\\ExcelData\\second_layer_stakeholder.xlsx'

In [7]:
pj = pd.read_csv(project, sep = ",")
risk1 = pd.read_excel(risk)
stk = pd.read_csv(stake, sep = ",")

prj = pd.read_excel(project1)
risk2 = pd.read_excel(risk0)
stk1 = pd.read_excel(stake1)
stk2 = pd.read_excel(stake2)
stk2.dropna()

en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

C:\Users\jqi22\AppData\Local\Temp\ipykernel_16576\2096727506.py:1: DtypeWarning: Columns (3,7,8,12,13,18,27,28,29,30,31,38,39,40,41,42,43,44,45,50,51,53,54,59,63,64,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  pj = pd.read_csv(project, sep = ",")


In [9]:
# get the target text from original dataset to match
nproject = pd.DataFrame(pj["Article Title"])
nrisk = pd.DataFrame(risk1["Abstract"])
nstack = pd.DataFrame(stk["Abstract"])

nrisk.Abstract = nrisk.Abstract.fillna("No Context")

stk1.name = stk1.name.str.lower()

In [11]:
def reload_dashes(droped: int)->pd.DataFrame:
    dashes = pd.read_excel(pathprefix+"\\adjustment.xlsx")
    dashes = dashes[dashes.frequency > droped].Words.to_list()
    return set(dashes)

stopwords |= reload_dashes(2)
# filout = pd.read_excel(r"D:\Code Working Area\Python\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData\filtered.xlsx")
# stopwords |= set(filout.name.to_list())
stopwords |= set([str(num) for num in range(1,100)])

# manually add the words to the stopwords
possiblew = {"connections", "efficacy", "life", "This"}
stopwords |= possiblew

In [175]:
# # split the stakeholder org words and count the frequency
# arr1 = stk1.name.str.split(" ").explode()
# fre = pd.DataFrame(data=arr1.value_counts())
# fre.reset_index(inplace = True)
# fre = fre.rename(columns = {"name": "frequency", "index": "name"})
# filtered = fre[fre.name.apply(lambda x: x not in stopwords)].reset_index(drop = True)
# filtered.head()

In [6]:
## pick up the ORG whose frequency larger than 30
# fre = pd.read_csv(r"D:\Code Working Area\Python\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData\Examples\ORG_frequency.csv")
# chosenone = fre[fre.frequency >= 30]
# chosenone = chosenone[chosenone.name.apply(lambda x: x not in stopwords)].reset_index(drop = True)

In [11]:
def combination(keyw: str, extracted: list):
    extracted = list(set(extracted))
    temp = []
    for val in range(0, len(extracted)):
        if re.sub("\W", "", extracted[val]) not in stopwords: temp.append(extracted[val])
    del extracted
    return [ele+" "+ keyw for ele in temp] if len(temp) else []

def regex_match(keyword: str, args:str):
    temp = args
    lookbehind = rf"(?<=\b{keyword})(\W\W?\w+)"
    lookforward = rf"(\w+\W\W?)(?=\b{keyword}\s)"
    return combination(keyword, re.findall(lookforward, temp)) + combination(keyword, re.findall(lookbehind, temp))

In [12]:
# pre-process of matching, but only apply for risk keyword abstraction
def reduction(args: str, val: str):
    args = str(args)
    for item in args:
        if item in ["(", ")", "+"]: args = args.replace(item, " ")
    try:
        return re.search(rf"\b{args}\b", val) != None
    except Exception:
        print("the currently word is: %s", args, flags = re.IGNORECASE)
    
def dummy_project(args: pd.Series, val: str):
    ags = str(args)
    return val.find(args) != -1

def match_attributes(args: str):
    """args are the value from nrisk.Abstract"""
    res = prj["Article Title"].apply(dummy_project, args = (args, ))
    casualty = prj["Article Title"][res == True].to_list()
    res = risk2["Abstract"].apply(reduction, args= (args, ))
    casualty = [*casualty, *risk2.Abstract[res==True].to_list()]
    # convert list into new rows (attention here the res is in pd.Series type not pd.DataFrame)
    res=stk2["stk"].apply(reduction, args=(args,))
    res=stk2.stk[res==True]
    casualty = [*casualty, *res.to_list()]
    return casualty

def write_json(new_data, filepath=jsfile):
    with open(filepath,'r+') as file:
          # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

# first run the first 500 lines and find out the frequency, delete the words whose frequency larger than 5
def extraction(rnum: tuple = (0, 500)):
    for value in nrisk.iloc[rnum[0]:rnum[1]].itertuples(): 
        if type(value.Abstract) is int or type(value.Abstract) is float: 
            print(value)
            continue
        write_json(match_attributes(value.Abstract))
#         match_attributes(value.Abstract)
    return

interval: list=[(0, 50), (150, 400), (1000, 1500)]

for items in interval:
    extraction(items)
# nrisk.value_counts

In [9]:
stk2

,0,1
0,NaN,stk
1,0.0,Stakeholder
2,1.0,the Stakeholder Circle Methodology
3,2.0,PPM
4,3.0,Research & Development
...,...,...
20478,20480.0,the Sustainable Human Resource Management
20479,20481.0,OLCA
20480,20482.0,The New York Times
20481,20483.0,Bi-Oceanic


In [6]:
# process on the stake keyword -- clean
files: list = []
with open(jsfile, "r") as file:
    files = json.load(file)
# files = pd.DataFrame({"Words": files})
# files["expansion"] = files.explode("Words").reset_index(drop = True)

# # sns.histplot(data = files, x = "Words", kde = True) # quiet useless
# dashes = pd.DataFrame(files.value_counts()).reset_index(level = 0)
# dashes.columns = ["words", "frequency"]

In [10]:
files=[x for x in files if x]
files[0:6]

[['uses risk', 'accident risk', 'operation risk', 'TAR'],
 ['safety risk'],
 ['early risk', 'aggregate risk'],
 ['different risk', 'Weibull'],
 ['construction project', 'struction project', 'political risk', 'CFA'],
 ['quantified risk',
  'proposed risk',
  'drought risk',
  'annual risk',
  'fed',
  'CC',
  'Correlation',
  'Curve']]

In [134]:
# files.to_csv(r"D:\Code Working Area\Python\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData\Examples\NewExample.csv", index = False)
# auxilary function, used to filter out stakeholder
stk1 = pd.read_excel(stake1)
def spacy_render(args: str):
    doc = nlp(args)
    displacy.render(doc, style = "ent")

def quardoric(args: str):
    doc=nlp(args)
    return [tect.text for tect in doc.ents if tect.label_ == "ORG"]
stk1.name = stk1.name.astype("object")

In [27]:
# in this step, we are going to handle on fp_growth
# fpgrowth minSupRatio should not over than 0.2
